In [1]:
import pandas as pd
from my_functions import *

In [2]:
# clean_matching()

In [3]:
# create_pegmed()

In [4]:
# create_pegprod()

In [5]:
# create_medcoded_entries()

In [6]:
# create_consultations()

In [7]:
# create_all_entries()

In [8]:
# create_pt_features()

In [9]:
# add_sys_start_and_end_dates_to_pt_features()

In [10]:
# add_length_of_data_pre_and_post_indexdate_to_pt_features(isCase=True)

In [11]:
# delete_patients_if_not_enough_data(isCase=True)

In [12]:
# match_cases_and_controls()

In [13]:
# delete_unmatched_controls()

In [14]:
# add_length_of_data_pre_and_post_indexdate_to_pt_features(isCase=False)

In [15]:
# delete_patients_if_not_enough_data(isCase=False)

In [16]:
# create_insomnia_entries()

In [17]:
# add_insomnia_event_count_to_pt_features()